In [3]:
import re
import datetime
import scipy
import sys
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import seaborn as sns
from scipy.signal import savgol_filter
from scipy.optimize import minimize_scalar
import scipy.linalg as linalg


from sys import stdout
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler, Binarizer, MinMaxScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split, cross_val_predict,cross_val_score , KFold
from sklearn.metrics import mean_squared_error, r2_score, make_scorer, explained_variance_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_predict, cross_val_score, KFold , \
    cross_validate, StratifiedKFold


import warnings
warnings.filterwarnings("ignore")

In [9]:
df = pd.read_csv("./data/data_tieu_hat_full.csv")
df.head(2)

,Code2,Code3,extract_date_from_filename,ID_machine,Temperature,Humidity,901.398183,905.331226,909.259927,913.184288,...,1673.98141,1676.942091,1679.898431,1682.85043,1685.798089,1688.741406,1691.680383,1694.615018,1697.545313,1701.44562
0,00893_36,Hadanard1_100800_20220623_141053.csv,6/23/2022,2,34.29,85.91,0.830749,0.807264,0.816415,0.812683,...,-0.172542,-0.189900,-0.200437,-0.196828,-0.179310,-0.169280,-0.156739,-0.154597,-0.150397,-0.126200
1,00893_36,Hadanard1_100800_20220623_141110.csv,6/23/2022,2,34.48,85.26,0.704169,0.701405,0.692683,0.690105,...,-0.069185,-0.090661,-0.098000,-0.096262,-0.084292,-0.072022,-0.062244,-0.054861,-0.055234,-0.031353


In [12]:
df['ID_machine'].value_counts()

ID_machine
1    21344
2    12185
Name: count, dtype: int64

In [13]:
wavelength = df.columns[6:]
wavelength = np.array(wavelength)
wavelength = wavelength.astype(float)

In [16]:
def D(xlist,ylist):
    yprime = np.diff(ylist)/np.diff(xlist)
    xprime=[]
    for i in range(len(yprime)):
        xtemp = (xlist[i+1]+xlist[i])/2
        xprime = np.append(xprime,xtemp)
    return xprime, yprime
plt.figure(1,dpi=120)
a = []
for i in range(len(df)):
    ylist = df.iloc[i,6:]
    xlist = wavelength
    xprime, yprime = D(xlist,ylist)
    if(yprime.max()>0.05 or yprime.min()< -0.05):
        a.append(i)
    plt.plot(xprime,yprime,label="Derivative")

In [17]:
print(len(a))
df = df.drop(index=a,inplace=False)
df = df.reset_index(drop=True)

121


In [28]:
df_machine1 = df[df['ID_machine'] == 1]
df_machine2 = df[df['ID_machine'] == 2]

In [29]:
a = df_machine2[df_machine2[df_machine2.iloc[:,6:] <= -0.8].any(axis=1)].index
df_machine2 = df_machine2.drop(index=a, axis=0)
df_machine2 = df_machine2.reset_index(drop=True)

In [30]:
X1 = np.array(df_machine1.iloc[:,6:].mean(axis=0))
X2 = np.array(df_machine2.iloc[:,6:].mean(axis=0))

In [31]:
calibration_factor = X2 / X1
calibrated_array1 = X1 * calibration_factor + (np.mean(X1) - np.mean(X2))

In [32]:
cross_corr = np.correlate(X1, X2, mode='same')
shift = np.argmax(cross_corr) - (len(X1) - 1)

In [33]:
for j in range(0,len(df_machine1)):
    df_machine1.iloc[j,6:] = df_machine1.iloc[j,6:] * calibration_factor

In [34]:
df2 = pd.concat([df_machine1, df_machine2])
df2 = df2.drop(columns=['Code3', 'Temperature', 'Humidity', 'ID_machine'])

In [35]:
result = df2.groupby(['Code2', 'extract_date_from_filename']).mean().reset_index()


In [36]:
result = result.replace({'02415-07':'02415_07', 
            '02422-09':'02422_09',
            '00389_02':'00398_02',
            '09625_15':'06925_15'})

In [37]:
result

,Code2,extract_date_from_filename,901.398183,905.331226,909.259927,913.184288,917.104308,921.019987,924.931325,930.13969,...,1673.98141,1676.942091,1679.898431,1682.85043,1685.798089,1688.741406,1691.680383,1694.615018,1697.545313,1701.44562
0,00072_22,6/22/2022,0.696805,0.694536,0.683930,0.681593,0.677989,0.672134,0.666920,0.665350,...,-0.244438,-0.268946,-0.284671,-0.290617,-0.284363,-0.280592,-0.270036,-0.262460,-0.259066,-0.235076
1,00098_20,5/10/2022,0.667177,0.663814,0.657104,0.651279,0.646839,0.640361,0.636948,0.633866,...,-0.088648,-0.109150,-0.118898,-0.118197,-0.105319,-0.097355,-0.082546,-0.074050,-0.071558,-0.048238
2,00104_19,5/17/2022,0.630155,0.622510,0.611900,0.606713,0.601914,0.596868,0.592414,0.590272,...,-0.076190,-0.094089,-0.102963,-0.103011,-0.092574,-0.085537,-0.072915,-0.065291,-0.063368,-0.042678
3,00107_14,6/9/2022,0.625186,0.624916,0.614951,0.612193,0.605344,0.601418,0.594491,0.591591,...,-0.069862,-0.085523,-0.092987,-0.092354,-0.082503,-0.075983,-0.064784,-0.057786,-0.056296,-0.037657
4,00108_03,7/7/2022,0.808258,0.804390,0.792780,0.789594,0.785070,0.778403,0.771624,0.768505,...,-0.045578,-0.066033,-0.076268,-0.075970,-0.063849,-0.055188,-0.040658,-0.031489,-0.027828,-0.004475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,08299_16,12/12/2021,0.680749,0.673351,0.660639,0.654679,0.650256,0.643894,0.639242,0.637389,...,-0.092796,-0.114902,-0.126290,-0.126314,-0.113391,-0.104725,-0.089061,-0.079786,-0.077034,-0.051824
253,08299_16,12/2/2021,0.753642,0.739299,0.723984,0.717217,0.713916,0.709636,0.706573,0.706251,...,-0.122311,-0.151144,-0.165913,-0.165535,-0.148216,-0.136146,-0.114798,-0.101526,-0.097192,-0.065011
254,08299_16,12/24/2021,0.573353,0.567305,0.557691,0.550972,0.543069,0.538508,0.536602,0.534776,...,-0.005326,-0.003238,-0.001982,-0.002717,-0.005295,-0.006462,-0.008055,-0.008413,-0.008808,-0.007515
255,08299_16,2/17/2022,0.704960,0.694961,0.684494,0.676237,0.671399,0.665302,0.661506,0.656345,...,-0.109429,-0.138010,-0.153614,-0.155249,-0.139786,-0.129676,-0.109677,-0.097881,-0.094165,-0.063077


In [45]:
df_hoaly = pd.read_excel("./data/Mã hóa mẫu tiêu.xlsx")
df_hoaly = df_hoaly[['Mã ghi trên túi', 'Nguồn gốc']]
df_hoaly

,Mã ghi trên túi,Nguồn gốc
0,00523_01,Đăk Lăk
1,00425_02,Đăk Lăk
2,00534_03,Đăk Lăk
3,00423_04,Đăk Lăk
4,00506_05,Đăk Lăk
...,...,...
153,02415_07,Gia Lai
154,02411_08,Gia Lai
155,02422_09,Gia Lai
156,02417_10,Gia Lai


In [51]:
df2['nguon_goc'] = pd.Series(dtype='int')

In [47]:
result.head(2)

,Code2,extract_date_from_filename,901.398183,905.331226,909.259927,913.184288,917.104308,921.019987,924.931325,930.13969,...,1676.942091,1679.898431,1682.85043,1685.798089,1688.741406,1691.680383,1694.615018,1697.545313,1701.44562,nguon_goc
0,00072_22,6/22/2022,0.696805,0.694536,0.683930,0.681593,0.677989,0.672134,0.666920,0.665350,...,-0.268946,-0.284671,-0.290617,-0.284363,-0.280592,-0.270036,-0.26246,-0.259066,-0.235076,NaN
1,00098_20,5/10/2022,0.667177,0.663814,0.657104,0.651279,0.646839,0.640361,0.636948,0.633866,...,-0.109150,-0.118898,-0.118197,-0.105319,-0.097355,-0.082546,-0.07405,-0.071558,-0.048238,NaN


In [52]:
for i in range(len(df_hoaly)):
    df2['nguon_goc'] = df2.apply(lambda x: df_hoaly['Nguồn gốc'][i] if ((x['Code2'] == df_hoaly['Mã ghi trên túi'][i])) else x['nguon_goc'], axis=1)

In [53]:
df2

,Code2,extract_date_from_filename,901.398183,905.331226,909.259927,913.184288,917.104308,921.019987,924.931325,930.13969,...,1676.942091,1679.898431,1682.85043,1685.798089,1688.741406,1691.680383,1694.615018,1697.545313,1701.44562,nguon_goc
764,02422_04,1/18/2022,0.578777,0.584949,0.531745,0.547919,0.524858,0.542858,0.515969,0.534316,...,-0.068244,-0.074027,-0.073144,-0.068114,-0.059003,-0.054677,-0.044159,-0.049387,-0.027969,Gia Lai
765,02422_04,1/18/2022,0.588914,0.567825,0.581933,0.558537,0.567248,0.543240,0.543247,0.545432,...,-0.049878,-0.051405,-0.052227,-0.045573,-0.042299,-0.034936,-0.030705,-0.030404,-0.021095,Gia Lai
766,02422_04,1/18/2022,0.527623,0.496573,0.463342,0.465414,0.468205,0.471219,0.448810,0.452861,...,-0.075449,-0.080974,-0.079504,-0.072765,-0.067486,-0.059119,-0.049621,-0.048991,-0.033799,Gia Lai
767,02422_04,1/18/2022,0.656137,0.616878,0.604265,0.595094,0.603415,0.600268,0.587448,0.578380,...,-0.082025,-0.088693,-0.088521,-0.079259,-0.075524,-0.062511,-0.056575,-0.052599,-0.036939,Gia Lai
768,02422_04,1/18/2022,0.741065,0.650167,0.632304,0.584572,0.611143,0.608705,0.617423,0.590556,...,-0.109028,-0.120232,-0.117477,-0.100585,-0.100661,-0.081412,-0.080739,-0.066877,-0.049152,Gia Lai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12152,09625_15,5/5/2023,0.619937,0.618583,0.613228,0.607720,0.608019,0.598505,0.594597,0.591941,...,-0.126126,-0.139913,-0.141143,-0.135085,-0.124642,-0.114095,-0.106025,-0.103977,-0.080253,NaN
12153,09625_15,5/5/2023,0.589767,0.601253,0.585968,0.592354,0.577790,0.574688,0.567730,0.566517,...,-0.133757,-0.145636,-0.154349,-0.144714,-0.139382,-0.126720,-0.127551,-0.122213,-0.094048,NaN
12154,09625_15,5/5/2023,0.502790,0.494467,0.483915,0.477713,0.474730,0.473691,0.469186,0.469291,...,-0.208395,-0.226177,-0.231729,-0.224332,-0.222073,-0.211278,-0.202526,-0.202876,-0.179269,NaN
12155,09625_15,5/5/2023,0.607810,0.584422,0.576773,0.559632,0.569531,0.566402,0.575821,0.562772,...,-0.082520,-0.098332,-0.102897,-0.109548,-0.092888,-0.076364,-0.046572,-0.067929,-0.056553,NaN


In [62]:
df3 = df2[pd.notna(df2['nguon_goc'])]

In [54]:
from sklearn.ensemble import RandomForestClassifier

In [79]:
X = df3.iloc[:,2:-1].values
y = df3.iloc[:,-1]

In [80]:
clf = RandomForestClassifier(random_state=0)
clf.fit(X, y)

RandomForestClassifier(random_state=0)

In [86]:
clf.predict(X[0].reshape(1, -1))

array(['Gia Lai'], dtype=object)

In [87]:
import pickle

In [88]:
# Save the model to a file
with open('./models/model_origin.pkl', 'wb') as f:
    pickle.dump(clf, f)